In [1]:
# importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx 
import random
import nest_asyncio
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from mesa.visualization.modules import ChartModule
from mesa.visualization.modules import NetworkModule
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.ModularVisualization import VisualizationElement

In [4]:
from Model import DiffusionModel, Household
seed1 = 44
nest_asyncio.apply()

########################## Define user interface ########################################

def network_portrayal(G):
    # The model ensures there is always 1 agent per node

    def node_color(agent):
        if agent.stage == 4:
            return "blue"
        elif agent.stage == 5:
            return "red"
        else:
            return "black"
        
    def node_size(agent):
        if agent.stage == 4:
            return 10
        elif agent.stage == 5:
            return 10
        else:
            return 6

    def edge_width(agent1, agent2):
        if agent1.time == 0:
            return 0.4
        else:
            return 0

    def get_agents(source, target):
        return G.nodes[source]['agent'][0], G.nodes[target]['agent'][0]

    portrayal = {}        
    portrayal['nodes'] = [{'size': node_size(agents[0]),
                           'color': node_color(agents[0]),
                            'tooltip': "stage: {}".format(agents[0].stage),
                           }
                          for (_, agents) in G.nodes.data('agent')]

    portrayal['edges'] = [{'source': source,
                           'target': target,
                           'color': "grey",
                           'width': edge_width(*get_agents(source, target)),
                           }
                          for (source, target) in G.edges]

    return portrayal

network = NetworkModule(network_portrayal, 727, 727, library='d3')
chart = ChartModule([{"Label": "Adoption",
                      "Color": "Black"}],
                    canvas_height=100,
                    canvas_width=300,
                    data_collector_name='datacollector')

model_params = {
    "seed": seed1,
    "price_tf": 1500,
    "price_cf": 6000,
    "p_imitators": 0,
    "p_optimizers": 0,
    "p_advice_seekers": 0,
    
    "cash_transfer": UserSettableParameter(
        "slider",
        "Cash transfer",
        0, 
        0, 
        4000,
        4000,
        description="Provide cash for fuel",
    ),
    "vouchers": UserSettableParameter(
        "slider",
        "Voucher",
        0, 
        0, 
        4000,
        4000,
        description="Provide vouchers for fuel",
    ),
    "info_campaign": UserSettableParameter(
        "slider",
        "Information campaign",
        0, 
        0, 
        5,
        5,
        description="Information campaign",
    ),
    "maintenance_capacity": UserSettableParameter(
        "slider",
        "Maintenance capacity",
        0, 
        0, 
        5,
        5,
        description="Increase maintenance capacity",
    ),
    "N": UserSettableParameter(
        "slider",
        "Number of agents",
        380,
        50,
        500,
        10,
        description="Choose how many agents to include in the model",
    ),
    "initial_adopters": UserSettableParameter(
        "slider",
        "Initial adopters",
        0.025,
        0,
        0.15,
        0.005,
        description="Initial adopters",
    ),
    "avg_node_degree": UserSettableParameter(
        "slider", 
        "Average node degree", 
        6, 4, 12, 1, 
        description="Average node degree"
    ),
    "p_rewiring": UserSettableParameter(
        "slider",
        "Probability of rewiring",
        0.2,
        0.1,
        0.9,
        0.1,
        description="Probability of rewiring",
    ),
    "r_discount": UserSettableParameter(
        "slider",
        "Time discount factor",
        0.1,
        0.0,
        0.1,
        0.025,
        description="Time discount factor for traditional fuel price",
    ),
    "atp": UserSettableParameter(
        "slider",
        "Ability-to-pay",
        0.025,
        0.0,
        0.05,
        0.005,
        description="Ability-to-pay",
    ),
    "p_bad_performance": UserSettableParameter(
        "slider",
        "Probability of bad performance",
        0.0,
        0.0,
        0.02,
        0.005,
        description="Probability that an agent experiences bad performance of clean cooking system",
    ),
    "p_supply_delay": UserSettableParameter(
        "slider",
        "Probability of supply shock",
        0.0,
        0.0,
        0.02,
        0.005,
        description="Probability that a supply shock occurs",
    ),
    "price_shock": UserSettableParameter(
        "slider",
        "Price shock magnitude",
        0.0,
        -0.02,
        0.02,
        0.01,
        description="Price shock magnitude [% of clean fuel price]",
    ),
    "n_cf_price_shock": UserSettableParameter(
        "slider",
        "Price shock frequency",
        0.0,
        0.0,
        6,
        1,
        description="Price shock frequency",
    ),
}

server = ModularServer(DiffusionModel,
                       [network, chart],
                       "Diffusion of clean cooking practices in refugee camps",
                        model_params,
                      )

#################### Choose a new port (!) and launch the user interface ######################

server.port = 8556
server.launch()